# Trial Module

- __author__ = 'BlackDChase,MR-TLL'
- __version__ = '0.4.2'
- [Imports](#Imports)
- [Hyperparameter](#Hyperparameters)
- [Training A3C](#Training-A3C)
- [Master Agent](#Master-Agent)
- [Train and Save](#Train-and-Save)
- [Testing Model](#Testing-Model)
- [Train](#Train)
- [Environment](#Environment)
- [Test](#Test)
- [Loading Agent](#Loading-Agent)
- [Plotting](#Plotting)

# Imports

In [ ]:
from lstm import LSTM
from env import LSTMEnv as ENV
#from TempEnv import TempEnv as ENV
from Agent import GOD
import log
import sys
import torch
from torch import nn, multiprocessing, device, Tensor
import matplotlib.pyplot as plt
import sys,os

# Hyperparameters

In [ ]:
"""
Hyperparameters
"""
stateSize = 13
log.info(f"stateSize = {stateSize}")
keywords={
        "n":6,
        "e":400,
        "t":30,
        "a":5,
        "s":1000,
        "d":False,
        "alr":1e-3,
        "clr":1e-3,
        "p":None
    }
stateSize = 13
log.info(f"stateSize = {stateSize}")

"""
Making the action space
"""
n=int(keywords['a']//2)
if keywords['a']%2==0:
    actionSpace = [i for i in range(-n*2,n*2+1,2)]
    actionSpace.pop(len(actionSpace)//2)
else:
    actionSpace = [i/10 for i in range(-n*25,n*25+1,25)]
keywords['a']=actionSpace

In [ ]:
actionSpace

# Training A3C

# Master Agent

In [ ]:
#""" For Training
god = GOD(
    stateSize=int(stateSize),
    actionSpace=keywords["a"],
    debug=keywords["d"],
    maxEpisode=int(keywords["e"]),
    nAgent=int(keywords["n"]),
    trajectoryLength=int(keywords["t"]),
    alr=keywords["alr"],
    clr=keywords["clr"]
)
print("Master Agent Made")
log.info("GOD inititated")
log.info(f"State Size = {int(stateSize)}")
log.info(f"Action Space = {keywords['a']}")
log.info(f"Debug = {keywords['d']}")
log.info(f"Max Episode = {int(keywords['e'])}")
log.info(f"Number of Agent = {int(keywords['n'])}")
log.info(f"Trajectory Length = {int(keywords['t'])}")
log.info(f"Actor Learning Rate = {keywords['alr']}")
log.info(f"Critic Learning Rate = {keywords['clr']}")
#"""

# Train and Save

In [ ]:
god.train()
god.saveModel("../Saved_model")

# Testing Model

In [ ]:
folder="../Saved_model/" 
folder=!echo $folder$(ls $folder -Art | grep 'Olog' | tail -n 1)
folder=folder[0]
p=!echo "$folder$(ls $folder | grep 'CritcModel.pt')" | rev | cut -b 14- | rev
keywords['p']=p[0]
# To Load Newly saved Model

# To crewate a new LOG
import log

# Loading Agent

In [ ]:
#""" For Testing
god = GOD(
    debug=keywords["d"],
    stateSize=int(stateSize),
    actionSpace=keywords["a"],
    path=keywords["p"],
)
print("Master Agent Made")
log.info("GOD inititated")
log.info(f"State Size = {int(stateSize)}")
log.info(f"Action Space = {keywords['a']}")
log.info(f"Debug = {keywords['d']}")
log.info(f"Path of Model = {keywords['p']}")
log.info(f"Steps tested for = {keywords['s']}")
#"""

# Enviornment

In [ ]:
#"""# Using LSTM Model as ENV
output_size = 13
input_dim = output_size
hidden_dim = 128
layer_dim = 1
model = LSTM(output_size, input_dim, hidden_dim, layer_dim,debug=keywords["d"])
model.loadM("ENV_MODEL/lstm_model.pt")
log.info(f"LSTM Model = {model}")
#env=ENV(model,"../Dataset/13_columns.csv")
env=ENV(
    model=model,
    dataset_path="../datasets/normalized_weird_13_columns_with_supply.csv",
    actionSpace=actionSpace,
    debug=keywords["d"],
)
env.reset()
print("Environment inititated")
"""
env = ENV(stateSize,actionSpace)
#"""
log.info("Environment inititated")

god.giveEnvironment(env)
log.info("Environment parsed, Boss inititated")

# Test

In [ ]:
"""
for 1000 timesteps
"""
time=keywords['s']
a3cStates = god.test(time=time)
normalStates = god.getNormalStates(time=time)
a3cProfit,normalProfit,diff = god.compare(a3cState=a3cStates,normalState=normalStates)

# Plotting

In [ ]:
fig,ax = plt.subplots(dpi=100)
ax.set_xlabel(f"Time step")

color='r'
ax.plot(normalProfit,color=color)
ax.tick_params(axis='y',labelcolor=color)
ax.set_ylabel('Profit w/o A3C',color=color)

ax1 = ax.twinx()
color='b'
ax1.plot(a3cProfit,color=color)
ax1.tick_params(axis='y',labelcolor=color)
ax1.set_ylabel('A3C Profit',color=color)

fig.tight_layout()
plt.show()

# Comparison

In [ ]:
fig,ax = plt.subplots(dpi=100)
ax.set_xlabel(f"Time step")
ax.plot(diff)
ax.set_ylabel(f"A3C Profit - Normal Profit")
fig.tight_layout()
plt.show()